In [4]:
!pip install yadisk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.6 MB/s eta 0:00:00


In [5]:
import yadisk
import os
from tqdm import tqdm
import sys
import openpyxl
import pandas as pd
import re
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import matplotlib.pyplot as plt

In [6]:
#removing warnings
import warnings
warnings.filterwarnings("ignore")

In [7]:
#setting display option
pd.set_option('display.precision', 2)

### Download files from yandeks disk

In [8]:
app_id = '...'
secret_id = '...'

In [9]:
y = yadisk.YaDisk(app_id, secret_id)
url = y.get_code_url()
print("Go to the following url: %s" % url)
code = input("Enter the confirmation code: ")

try:
    response = y.get_token(code)
except yadisk.exceptions.BadRequestError:
    print("Bad code")
    sys.exit(1)
y.token = response.access_token
if y.check_token():
    print("Connected.")
else:
    print("Something wrong.")

Go to the following url: https://oauth.yandex.ru/authorize?response_type=code&client_id=734e4c9f1eab43f8b6526a157f6a7ee6&display=popup&force_confirm=yes
Enter the confirmation code: 7369026
Connected.


In [10]:
#collecting the list of files to download
list_of_files = []
for item in list(y.listdir('file_processor')):
    if item['path'].endswith('.xlsx') or item['path'].endswith('.json'):
        list_of_files.append(item['path'])

In [11]:
#creating folder to put files in and going there
load_path = '/content/lab'
if not os.path.exists(load_path):
    os.mkdir(load_path)
os.chdir(load_path)

In [12]:
#downloading files from yandex disk
for file in tqdm(list_of_files):
  if file.split('/')[-1] not in os.listdir():
    y.download(file.split(':')[1], file.split('/')[-1])

100%|██████████| 148/148 [05:00<00:00,  2.03s/it]


### Splitting files into 4 lists, which will be used to concatenate tables.

In [13]:
#functions check what is in the first rows of tables

#checking if there is target string in the fist rows of table
def check_string_in_first_rows(worksheet, target_string):
    for row in worksheet.iter_rows(min_row=0, max_row=5):
        for cell in row:
            if cell.value and target_string.upper() in str(cell.value).upper():
                return True
    return False

#returning the list of tables that contain the target string
def check_first_rows(list, target_string):
    list_of_tables = []
    for name in list:
      file_path = f'/content/lab/{name}'
      workbook = openpyxl.load_workbook(file_path, read_only=True)
      worksheet = workbook.worksheets[0]

      if check_string_in_first_rows(worksheet, target_string):
          list_of_tables.append(file_path.split('/')[-1])

    return list_of_tables

In [14]:
#splitting into lists by file name
sop_list, zno_list = [], []
for name in os.listdir():
  if 'состояние_онко_помощи' in name.lower():
    sop_list.append(name)
  if 'злокачественные_новообразования' in name.lower():
      zno_list.append(name)

In [15]:
#splitting into lists by first rows of tables
target_strings_sop = ['лечении злокачественных', 'контингенте']
target_strings_zno = ['заболеваемость населения', 'смертность населения']

sop_treatment = check_first_rows(sop_list, target_strings_sop[0])
sop_population = check_first_rows(sop_list, target_strings_sop[1])
zno_incidence = check_first_rows(zno_list, target_strings_zno[0])
zno_mortality = check_first_rows(zno_list, target_strings_zno[1])

### All functions

In [16]:
#function to extract data from the first cell of Состояние_онко_помощи table to collect data for extra columns:
#year, desease localization, icd code, table number
def sop_new_cols(table):

    values = {}

    #open file
    workbook = openpyxl.load_workbook(table, read_only=True)
    worksheet = workbook.worksheets[0]

    #read first cell
    first_cell = worksheet.cell(row=1, column=1).value

    #year
    year_match = re.search(r'\b\d{4}\b', first_cell)
    year = year_match.group(0) if year_match else None
    values.update({'year': year})

    #localization
    name_match = re.search(r'\n(.*?)(\(C\d+|\(С\d+)', first_cell)
    name = name_match.group(1).strip() if name_match else None
    values.update({'localization': name})

    #icd code
    code_match = re.search(r'\((C[^)]+|С[^)]+)\)', first_cell)
    code = code_match.group(1) if code_match else None
    values.update({'icd_code': code})

    #table number
    table_match = re.search(r'Таблица (\d+)', first_cell)
    table = table_match.group(1) if table_match else table.split('_')[2]
    values.update({'table': table})

    return values

In [17]:
#function to extract data from the first cell of Злокачественные_новообразования table to collect data for extra columns:
#year, desease localization, icd code, table number
def zno_new_cols(table):

    values = {}

    workbook = openpyxl.load_workbook(table, read_only=True)
    worksheet = workbook.worksheets[0]

    if worksheet['B3'].value is None:
      year_match = re.search(r'\d{4}$', worksheet['A3'].value)
      year = year_match.group(0).strip() if year_match else None
    else:
      year = worksheet['B3'].value
    values.update({'year': year})

    if worksheet['B4'].value is None:
      name_match = re.search(r': (.*?)(\(C\d+|\(С\d+)', worksheet['A4'].value)
    else:
      name_match = re.search(r'(.*?)(\(C\d+|\(С\d+)', worksheet['B4'].value)
    name = name_match.group(1).strip() if name_match else worksheet['B4'].value.strip()
    values.update({'localization': name})

    if worksheet['B4'].value is None:
      code_match = re.search(r'\((C[^)]+|С[^)]+)\)', worksheet['A4'].value)
    else:
      code_match = re.search(r'\((C[^)]+|С[^)]+)\)', worksheet['B4'].value)
    code = code_match.group(1) if code_match else worksheet['B4'].value.strip()
    values.update({'icd_code': code})

    table_match = re.search(r'Таблица (\d+)', worksheet['A1'].value)
    table = table_match.group(1) if table_match else table.split('_')[2]
    values.update({'table': table})

    return values

In [18]:
#function to join columns multiindex and do some minor corrections
def column_names_change(df):
  df.columns = df.columns.map(lambda x: ' '.join(map(str, x))) \
                                          .str.replace('\n', ' ') \
                                          .str.replace(r'Unnamed: \d+_level_\d+|- |\.\d$', '', regex=True) \
                                          .str.strip()
  df.columns = [' '.join(s.split()).lower() for s in df.columns]
  df = df.rename(columns={df.columns[0]: 'республика, край, область'})
  return df.columns

In [19]:
#function to remove rows with aggregated data
def remove_aggregated_lines(df):
  df = df[~df.iloc[:, 0].str.contains('россия', case=False) & \
          ~df.iloc[:, 0].str.contains('ФО')] \
          .reset_index(drop=True)
  return df

### Building the dataset using data from tables Состояние онко помощи (лечение).

In [20]:
#read first table
data_sop_treatment = pd.read_excel(sop_treatment[0], header=[1,2])

#create new columns with year, localization of desease, idc code, table number
new_cols = sop_new_cols(sop_treatment[0])
for key, value in new_cols.items():
    data_sop_treatment[key] = value

In [21]:
#build the dictionary: each region belongs to what federal district
fo_dict = {}
current_fo = 0

for item in data_sop_treatment.iloc[:, 0]:
  if item.endswith('ФО'):
    current_fo = item
  else:
    fo_dict.update({item: current_fo})

In [22]:
#read and concatenate the rest of tables from sop_treatment list
for file in sop_treatment[1:]:
    df = pd.read_excel(file, header=[1,2])
    new_cols = sop_new_cols(file)
    for key, value in new_cols.items():
        df[key] = value
    data_sop_treatment = pd.concat([data_sop_treatment, df])

In [23]:
#correct the columns names
data_sop_treatment.columns = column_names_change(data_sop_treatment)

In [24]:
#remove rows with aggregated data: grouped by federal district and country
data_sop_treatment = remove_aggregated_lines(data_sop_treatment)

In [25]:
#adding column with federal district for every region
data_sop_treatment['federal_district'] = data_sop_treatment.iloc[:, 0].map(fo_dict)

In [26]:
#correcting types, checking result
data_sop_treatment['year'] = data_sop_treatment['year'].astype(int)
data_sop_treatment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2202 entries, 0 to 2201
Data columns (total 15 columns):
 #   Column                                                                                              Non-Null Count  Dtype  
---  ------                                                                                              --------------  -----  
 0   республика, край, область                                                                           2202 non-null   object 
 1   число зно, выявленных в отчетном году, радикальное лечение которых закончено в отчетном году        2202 non-null   int64  
 2   число зно, выявленных в отчетном году, радикальное лечение которых % от впервые выявленных          2202 non-null   float64
 3   число зно, выявленных в отчетном году, радикальное лечение которых будет продолжено (не закончено)  2202 non-null   int64  
 4   число зно, выявленных в отчетном году, радикальное лечение которых % от впервые выявленных          2202 non-null 

### Building the dataset using data from tables Состояние онко помощи (контингент).

In [27]:
#read first table
data_sop_population = pd.read_excel(sop_population[0], header=[1,2])

#create new columns with year, localization of desease, idc code, table number
new_cols = sop_new_cols(sop_population[0])
for key, value in new_cols.items():
    data_sop_population[key] = value

In [28]:
#read and concatenate the rest of tables from sop_population list
for file in sop_population[1:]:
    df = pd.read_excel(file, header=[1,2])
    new_cols = sop_new_cols(file)
    for key, value in new_cols.items():
        df[key] = value
    data_sop_population = pd.concat([data_sop_population, df])

In [29]:
#remove rows with aggregated data: grouped by federal district and country
data_sop_population = remove_aggregated_lines(data_sop_population)

In [30]:
#adding column with federal district for every region
data_sop_population['federal_district'] = data_sop_population.iloc[:, 0].map(fo_dict)

In [31]:
#correct the columns names
data_sop_population.columns = column_names_change(data_sop_population)

In [32]:
#correcting types, checking result
data_sop_population['year'] = data_sop_population['year'].astype(int)
data_sop_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2380 entries, 0 to 2379
Data columns (total 14 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   республика, край, область                                   2380 non-null   object 
 1   взято на учет больных с впервые в жизни уст. диагнозом зно  2380 non-null   int64  
 2   в т.ч. выявлены активно, %                                  2380 non-null   float64
 3   находились на учете на конец года абсолютное число          2380 non-null   int64  
 4   находились на учете на конец года на 100 тыс. населения     2380 non-null   float64
 5   из них 5 лет и более абсолютное число                       2380 non-null   int64  
 6   из них 5 лет и более % от сост. на учете                    2380 non-null   float64
 7   индекс накопления контингентов                              2380 non-null   float64
 8 

### Building the dataset using data from tables Злокачественные новообразования (заболеваемость и смертность)

In [33]:
#function to split the table into 3 tables based on sex and concatenate them
#do not split tables of deseases that are applicable to one sex only
#create new columns with year, localization of desease, idc code, table number
def process_table(table):

  workbook = openpyxl.load_workbook(table, read_only=True)
  num_columns = workbook.worksheets[0].max_column

  #condition checks if the desease is applicable to all sexes or not
  if num_columns > 10:
    #read the table
    df = pd.read_excel(table, header=[5, 6])
    #split based on sex, put all 3 into the list
    lst = []
    for i in range(0, 3):
      lst.append(df.iloc[:, [0] + [4*i + j + 1 for j in range(4)]])

    for i, item in enumerate(lst):
      #correct the columns names
      item.columns = column_names_change(item)

      #add the column with sex
      #find the sex in the table partition, add to the new column
      workbook = openpyxl.load_workbook(table, read_only=True)
      worksheet = workbook.worksheets[0]
      item['sex'] = worksheet.cell(row=5, column=4*i + 2).value

    #concatenate
    df = pd.concat([lst[0], lst[1], lst[2]])

  else:
    #read the table
    df = pd.read_excel(table, header=[4, 5])
    #correct the columns names
    df.columns = column_names_change(df)
    #add the column with sex
    df['sex'] = None

  #create new columns with year, localization of desease, idc code, table number
  new_cols = zno_new_cols(table)
  for key, value in new_cols.items():
      df[key] = value

  return df

In [34]:
#read the first table
data_zno = process_table(zno_incidence[0])
data_zno['category'] = 'заболеваемость'

In [35]:
#read and concatenate the rest of tables from zno_incidence and zno_mortality lists
for i, lst in enumerate([zno_incidence[1:], zno_mortality]):
  for file in tqdm(lst):
    data_zno = pd.concat([data_zno, process_table(file)])
  if i == 0:
    data_zno['category'] = data_zno['category'].fillna('заболеваемость')
  else:
    data_zno['category'] = data_zno['category'].fillna('смертность')

100%|██████████| 35/35 [00:57<00:00,  1.64s/it]


In [36]:
#remove rows with aggregated data: grouped by federal district and country
data_zno = remove_aggregated_lines(data_zno)

In [37]:
#adding column with federal district for every region
data_zno['federal_district'] = data_zno.iloc[:, 0].str.strip().map(fo_dict)

In [38]:
#correcting types
data_zno['year'] = data_zno['year'].astype(int)

In [39]:
#checking the tables that didn't contain information about sex
data_zno.query('sex.isna()')['localization'].unique()

array(['Вульва', 'Влагалище', 'Половой член', 'Тело матки', 'Яичник',
       'Шейка матки', 'Яичко', 'Предстательная железа', 'Плацента',
       'Другие новообразования матки', 'Другие женские половые органы',
       'Другие мужские половые органы'], dtype=object)

In [40]:
#creating dictionary that defines sex based on desease
sex_dictionary = {"Женщины": ["Влагалище", "Вульва", "Яичник", "Шейка матки", "Тело матки", "Плацента", "Другие женские половые органы", "Другие новообразования матки"]
                  , "Мужчины": ["Половой член", "Яичко", "Предстательная железа", "Другие мужские половые органы"]}

In [41]:
#applying to dataset
for key, value in sex_dictionary.items():

    condition = data_zno['localization'].isin(value)
    data_zno['sex'] = data_zno.apply(
        lambda row: key if pd.isna(row['sex']) and condition[row.name] else row['sex'],
        axis=1
    )

In [42]:
data_zno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18615 entries, 0 to 18614
Data columns (total 12 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   республика, край, область                           18615 non-null  object
 1   абсолютное число                                    18615 non-null  int64 
 2   показатель на 100 тыс. населения "грубый"           18615 non-null  object
 3   показатель на 100 тыс. населения стандартизованный  18615 non-null  object
 4   показатель на 100 тыс. населения ошибка             18615 non-null  object
 5   sex                                                 18615 non-null  object
 6   year                                                18615 non-null  int64 
 7   localization                                        18615 non-null  object
 8   icd_code                                            18615 non-null  object
 9   table 

### Building the dataset using data from tables Состояние онко помощи (вторая вкладка).

In [43]:
data_sop_second = pd.read_excel(sop_population[0], sheet_name=1, header=[1, 2, 3])

new_cols = sop_new_cols(sop_population[0])
for key, value in new_cols.items():
    data_sop_second[key] = value

data_sop_second.columns = column_names_change(data_sop_second)

In [44]:
for file in tqdm(sop_population[1:]):
    try:
      df = pd.read_excel(file, sheet_name=1, header=[1, 2, 3])
    except Exception:
      continue
    new_cols = sop_new_cols(file)
    for key, value in new_cols.items():
        df[key] = value
    df.columns = data_sop_second.columns
    data_sop_second = pd.concat([data_sop_second, df])

100%|██████████| 27/27 [00:34<00:00,  1.27s/it]


In [45]:
#removing empty lines that came from numbers of pages
data_sop_second = data_sop_second.dropna()

In [46]:
#removing aggregated rows
data_sop_second = remove_aggregated_lines(data_sop_second)

In [47]:
#adding federal district
data_sop_second['federal_district'] = data_sop_second.iloc[:, 0].str.strip().map(fo_dict)

In [48]:
#correcting types
data_sop_second['year'] = data_sop_second['year'].astype(int)
data_sop_second.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2206 entries, 0 to 2205
Data columns (total 14 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   республика, край, область                              2206 non-null   object 
 1   зарегистрировано зно (без учтенных посмертно)          2206 non-null   float64
 2   из них диагноз подтвержден морфологически, %           2206 non-null   float64
 3   из них имели стадию заболевания, % i                   2206 non-null   object 
 4   из них имели стадию заболевания, % ii                  2206 non-null   object 
 5   из них имели стадию заболевания, % iii                 2206 non-null   object 
 6   из них имели стадию заболевания, % iv                  2206 non-null   object 
 7   из них имели стадию заболевания, % не установлена      2206 non-null   object 
 8   летальность на первом году с момента уст. диагно

### Uploading datasets to google sheets.

In [49]:
#connecting
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(f"{load_path}/onco-file-processor-dc07dc7f89c4.json", scope)
client = gspread.authorize(creds)

In [50]:
#creating dictionary with names of datasets and data
final_list = {"Oncology care status (treatment)": data_sop_treatment
              , "Oncology care status (population)": data_sop_population
              , "Oncology care status (population) table 2": data_sop_second
              , "Malignant neoplasms: incidence and mortality": data_zno}

In [51]:
for key, value in final_list.items():
  spreadsheet = client.create(key)
  worksheet = spreadsheet.sheet1
  worksheet.update([value.columns.values.tolist()] + value.values.tolist())
  spreadsheet.share("", perm_type="anyone", role="reader")
  print(f'{key}: {spreadsheet.url}')

Oncology care status (treatment): https://docs.google.com/spreadsheets/d/11ch2hH_jGhFBuNQslV2F_2b8GN374RS-c6bS2X7CnVs
Oncology care status (population): https://docs.google.com/spreadsheets/d/1P33EZT-CQqu1atFeqAsPWCy-Bp3rvw87iPRnJckTQcM
Oncology care status (population) table 2: https://docs.google.com/spreadsheets/d/1KKfeebttMfPreGBm1Zzm0ag5fBHLdcM9fgZLos2HQUc
Malignant neoplasms: incidence and mortality: https://docs.google.com/spreadsheets/d/1uj9oTNnAV_8FntB5Lsrw7Rskbdjc9uXepH0pVQzSR7M
